In [ ]:
! git clone https://github.com/akucsd/Latent_ODE_ECE_228.git

Cloning into 'Latent_ODE_ECE_228'...
remote: Enumerating objects: 2031, done.
remote: Counting objects: 100% (32/32), done.
remote: Compressing objects: 100% (25/25), done.
remote: Total 2031 (delta 13), reused 21 (delta 7), pack-reused 1999
Receiving objects: 100% (2031/2031), 55.00 MiB | 14.02 MiB/s, done.
Resolving deltas: 100% (1696/1696), done.


In [ ]:
# ! git clone https://github.com/openai/gym.git
# !pip install -e /content/gym
!pip install gym==0.22
!pip install free-mujoco-py
!pip install moviepy==1.0.0
# !pip install tensorboard==2.3.0 tensorboardX==1.8 matplotlib ipython==6.4.0 moviepy==1.0.0 pyvirtualdisplay==1.3.2
!pip install torch
!pip install mujoco
# !pip install opencv-python==4.4.0.42
!pip install ipdb==0.13.3
!pip install ray
!pip install tqdm
! pip install pyvirtualdisplay

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 631.1/631.1 kB 13.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for gym: filename=gym-0.22.0-py3-none-any.whl size=708362 sha256=978913fae7f1a59351192cc9d01a26eff7a39f30c42fe146ac699ca8dddb34f9
  Stored in directory: /root/.cache/pip/wheels/42/e8/e8/6dfbc92a1dcd76c1a5e2bb982750fd6b7e792239f46039e6b1
Successfully built gym
  Attempting uninstall: gym
    Found existing installation: gym 0.25.2
    Uninstalling gym-0.25.2:
      Successfully uninstalled gym-0.25.2
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 76.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 75.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 203.7/203.7 kB 28.5 MB/s eta 0:00:00
  Attempting uninstall: Cython
    Found existing installation: Cython 3.0.10
    Uninstalling Cython-3.0.10:
      Succ

In [ ]:
! git clone https://github.com/nslyubaykin/relax_dyna_q_example.git
! git clone https://github.com/nslyubaykin/relax.git

Cloning into 'relax_dyna_q_example'...
remote: Enumerating objects: 96, done.
remote: Counting objects: 100% (45/45), done.
remote: Compressing objects: 100% (45/45), done.
remote: Total 96 (delta 24), reused 4 (delta 0), pack-reused 51
Receiving objects: 100% (96/96), 95.76 MiB | 14.29 MiB/s, done.
Resolving deltas: 100% (24/24), done.
Cloning into 'relax'...
remote: Enumerating objects: 535, done.
remote: Counting objects: 100% (155/155), done.
remote: Compressing objects: 100% (113/113), done.
remote: Total 535 (delta 103), reused 79 (delta 42), pack-reused 380
Receiving objects: 100% (535/535), 198.21 KiB | 15.25 MiB/s, done.
Resolving deltas: 100% (333/333), done.


In [ ]:
!apt-get install -y \
    libgl1-mesa-dev \
    libgl1-mesa-glx \
    libglew-dev \
    libosmesa6-dev \
    software-properties-common

!apt-get install -y patchelf
!sudo apt-get install xvfb

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
software-properties-common is already the newest version (0.99.22.9).
The following additional packages will be installed:
  libegl-dev libgl-dev libgles-dev libgles1 libglu1-mesa libglu1-mesa-dev libglvnd-core-dev
  libglvnd-dev libglx-dev libopengl-dev libosmesa6
The following NEW packages will be installed:
  libegl-dev libgl-dev libgl1-mesa-dev libgl1-mesa-glx libgles-dev libgles1 libglew-dev
  libglu1-mesa libglu1-mesa-dev libglvnd-core-dev libglvnd-dev libglx-dev libopengl-dev libosmesa6
  libosmesa6-dev
0 upgraded, 15 newly installed, 0 to remove and 45 not upgraded.
Need to get 4,020 kB of archives.
After this operation, 19.4 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/main amd64 libglx-dev amd64 1.4.0-1 [14.1 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/main amd64 libgl-dev amd64 1.4.0-1 [101 kB]
Get:3 http://archive.ubuntu.com/ubuntu 

In [ ]:
%cd /content/relax
import gym
import torch
import numpy as np
import matplotlib.pyplot as plt

from tqdm import tqdm
from inspect import getsource
from torch.utils.tensorboard import SummaryWriter

from relax.rl.actors import TD3, RandomUniform
from relax.rl.critics import CDQN
from relax.rl.models import DeltaEnvModel

from relax.zoo.policies import DeterministicMLP
from relax.zoo.critics import ContQMLP
from relax.zoo.models import ContObsContAcsToObsMLP, ContObsContAcsToRewsMLP

from relax.schedules import PiecewiseSchedule, LinearSchedule, CombinedSchedule
from relax.exploration import RandomNormal

from relax.data.sampling import Sampler
from relax.data.replay_buffer import ReplayBuffer

# from relax.gym.utils import visualize_actor
from relax.gym.terminal_functions import walker2d_v2_terminal_fn

/content/relax


In [ ]:
def get_model(obs_dim, acs_dim,
              terminal_function,
              ensemble_size=7):

    obs_models = []
    rews_models = []
    for i in range(ensemble_size):
        obs_models.append(
            ContObsContAcsToObsMLP(input_obs_dim=obs_dim, output_obs_dim=obs_dim,
                                   acs_dim=acs_dim, nunits=250, nlayers=2) #350 for ant
        )
        rews_models.append(
            ContObsContAcsToRewsMLP(input_obs_dim=obs_dim, acs_dim=acs_dim,
                                    nunits=250, nlayers=2)
        )

    # Creating schedules:
    # first 5000 iterations no learning - just data collection
    model_lr = PiecewiseSchedule({0: 5000}, 1e-3)

    model = DeltaEnvModel(
        obs_models=obs_models,
        device=torch.device('cuda'),
        learning_rate=model_lr,
        noise=0.01,
        batch_size=512,
        rews_models=rews_models,
        terminal_function=terminal_function,
        n_steps_per_update=1,
        update_freq=10,
        stats_recalc_freq=500,
        random_agg=False,
    )

    return model

In [ ]:
def get_actor_and_critic(obs_dim, acs_dim):

    # Creating schedules:
    # first 10000 iterations no learning - just data collection
    policy_lr = PiecewiseSchedule({0: 10000}, 1e-3)
    critic_lr = PiecewiseSchedule({0: 10000}, 1e-3)

    schedule1 = PiecewiseSchedule({0.3: 10000}, 0.0)
    schedule2 = LinearSchedule(0.1, 0, 1000000)
    sigma_schedule = CombinedSchedule(schedule1, schedule2, max)

    # Define exploration
    exploration = RandomNormal(sigma=sigma_schedule,
                               min_acs=-1.0,
                               max_acs=1.0,
                               n_random_steps=10000)

    actor = TD3(
        device=torch.device('cuda'),
        mu_net=DeterministicMLP(obs_dim, acs_dim, out_activation=torch.nn.Tanh()),
        learning_rate=policy_lr,
        batch_size=100,
        min_acs=-1.0,
        max_acs=1.0,
        exploration=exploration,
    )

    critic = CDQN(
        critic_net=ContQMLP(obs_dim, acs_dim),
        critic_net2=ContQMLP(obs_dim, acs_dim),
        device=torch.device('cuda'),
        learning_rate=critic_lr,
        weight_decay=0.0
    )

    return actor, critic


In [ ]:
env_name = 'Walker2d-v2'
terminal_function = walker2d_v2_terminal_fn

acs_dim = 6
obs_dim = 17
n_experiments = 4
n_steps = 1000000

In [ ]:
%cd ..
%cd relax_dyna_q_example

/content
/content/relax_dyna_q_example


In [ ]:

actor, critic = get_actor_and_critic(obs_dim=obs_dim, acs_dim=acs_dim)

# Model
model = get_model(obs_dim=obs_dim, acs_dim=acs_dim,
                  terminal_function=terminal_function,
                  ensemble_size=7)

# provide actor with critic
actor.set_critic(critic)

# setting the schedule for acceleration horizon
horizon_schedule = PiecewiseSchedule({0: 20000}, 1)

# provide actor with model for acceleration
actor.set_acceleration(model=model,
                       actor=RandomUniform(acs_dim=acs_dim,
                                           min_acs=-1,
                                           max_acs=1),
                       h=horizon_schedule,
                       tau=3.0,
                       real_ratio=0.0,
                       train_sampling=True)

# Saving models:
actor.load_checkpoint('trained_models/dyna_q_td3', 'dyna_q_td3_Walker2d-v2_experiment_2_actor')
critic.load_checkpoint('trained_models/dyna_q_td3', 'dyna_q_td3_Walker2d-v2_experiment_2_critic')
model.load_checkpoint('trained_models/dyna_q_td3', 'dyna_q_td3_Walker2d-v2_experiment_2_model')

Loaded checkpoints for TD3...
models_state_dict exploration_global_step global_step local_step n_updates n_target_updates optimizer n_policy_updates
Loaded checkpoints for CDQN...
models_state_dict global_step local_step n_updates n_target_updates optimizer optimizer2
Loaded checkpoints for DeltaEnvModel...
models_state_dict global_step local_step n_updates n_stats_updates n_model_resets updates_per_model obs_optimizer_0 obs_optimizer_1 obs_optimizer_2 obs_optimizer_3 obs_optimizer_4 obs_optimizer_5 obs_optimizer_6 rews_optimizer_0 rews_optimizer_1 rews_optimizer_2 rews_optimizer_3 rews_optimizer_4 rews_optimizer_5 rews_optimizer_6 buffer_stats


In [ ]:
env_name = 'Walker2d-v2'
eval_env = gym.make(env_name)
eval_env.seed(np.random.randint(100))
eval_sampler = Sampler(env=eval_env)

/usr/local/lib/python3.10/dist-packages/gym/envs/registration.py:505: UserWarning: WARN: The environment Walker2d-v2 is out of date. You should consider upgrading to version `v3` with the environment ID `Walker2d-v3`.
  logger.warn(


Compiling /usr/local/lib/python3.10/dist-packages/mujoco_py/cymj.pyx because it changed.
[1/1] Cythonizing /usr/local/lib/python3.10/dist-packages/mujoco_py/cymj.pyx


INFO:root:running build_ext
INFO:root:building 'mujoco_py.cymj' extension
INFO:root:creating /usr/local/lib/python3.10/dist-packages/mujoco_py/generated/_pyxbld_2.0.2.13_310_linuxcpuextensionbuilder
INFO:root:creating /usr/local/lib/python3.10/dist-packages/mujoco_py/generated/_pyxbld_2.0.2.13_310_linuxcpuextensionbuilder/temp.linux-x86_64-cpython-310
INFO:root:creating /usr/local/lib/python3.10/dist-packages/mujoco_py/generated/_pyxbld_2.0.2.13_310_linuxcpuextensionbuilder/temp.linux-x86_64-cpython-310/usr
INFO:root:creating /usr/local/lib/python3.10/dist-packages/mujoco_py/generated/_pyxbld_2.0.2.13_310_linuxcpuextensionbuilder/temp.linux-x86_64-cpython-310/usr/local
INFO:root:creating /usr/local/lib/python3.10/dist-packages/mujoco_py/generated/_pyxbld_2.0.2.13_310_linuxcpuextensionbuilder/temp.linux-x86_64-cpython-310/usr/local/lib
INFO:root:creating /usr/local/lib/python3.10/dist-packages/mujoco_py/generated/_pyxbld_2.0.2.13_310_linuxcpuextensionbuilder/temp.linux-x86_64-cpython-31

In [ ]:
import glob
import io
import base64
import numpy as np

from gym.wrappers import Monitor
from gym import Wrapper
from IPython.display import HTML
from IPython import display as ipythondisplay
from pyvirtualdisplay import Display

from relax.data.sampling import Sampler



In [ ]:
def wrap_env(env):
    env = Monitor(env, 'content/video', force=True)
    return env

def show_video_func():
    mp4list = glob.glob('content/video/*.mp4')
    if len(mp4list) > 0:
        mp4 = mp4list[0]
        video = io.open(mp4, 'r+b').read()
        encoded = base64.b64encode(video)
        ipythondisplay.display(HTML(data='''<video alt="test" autoplay
                    loop controls style="height: 400px;">
                    <source src="data:video/mp4;base64,{0}" type="video/mp4" />
                 </video>'''.format(encoded.decode('ascii'))))
    else:
        print("Could not find video")

def set_random_initial_state(smp,incline_angle=0, case_number=1):
    env = smp.env
    env.reset()
    # Assuming the environment is of type 'Walker2d-v3' with specific qpos and qvel attributes
    qpos = env.env.sim.data.qpos
    qvel = env.env.sim.data.qvel
    # print(qpos)
    # print(qvel)

    if case_number == 1:
      qpos[1:2] =  qpos[1:2]+ np.random.uniform(low=-0.005, high=0.005, size=1)  # Randomize root position (excluding x-axis)
      qpos[2:] = qpos[2:] + np.random.uniform(low=-0.005, high=0.005, size=qpos[2:].shape)  # Randomize joint positions
      qvel[:] = qvel[:] + np.random.uniform(low=-0.005, high=0.005, size=qvel.shape)  # Randomize velocities

    if case_number == 2:
      qpos[1:2] =  qpos[1:2]+ np.random.uniform(low=-0.05, high=0.05, size=1)  # Randomize root position (excluding x-axis)
      qpos[2:] = qpos[2:] + np.random.uniform(low=-0.087, high=0.087, size=qpos[2:].shape)  # Randomize joint positions
      qvel[:] = qvel[:] + np.random.uniform(low=-0.05, high=0.05, size=qvel.shape)  # Randomize velocities

    if case_number == 3:
      qpos[1:2] =  qpos[1:2]+ np.random.uniform(low=-0.1, high=0.1, size=1)  # Randomize root position (excluding x-axis)
      qpos[2:] = qpos[2:] + np.random.uniform(low=-0.17, high=0.17, size=qpos[2:].shape)  # Randomize joint positions
      qvel[:] = qvel[:] + np.random.uniform(low=-0.1, high=0.1, size=qvel.shape)  # Randomize velocities

    if case_number == 4:
      qpos[1:2] =  qpos[1:2]+ np.random.uniform(low=-0.15, high=0.15, size=1)  # Randomize root position (excluding x-axis)
      qpos[2:] = qpos[2:] + np.random.uniform(low=-0.26, high=0.26, size=qpos[2:].shape)  # Randomize joint positions
      qvel[:] = qvel[:] + np.random.uniform(low=-0.1, high=0.1, size=qvel.shape)  # Randomize velocities

    # print(qpos)
    # print(qvel)

    # env.env.set_state(qpos, qvel)
    ob = np.concatenate([qpos[1:], qvel])  # Exclude the first dimension (x-axis) of qpos
    smp.add_obs(ob)

    # Set the inclination of the plane
    model = env.env.model
    model.opt.gravity[0] = 9.81 * np.sin(incline_angle)  # Adjust gravity for inclination
    model.opt.gravity[2] = -9.81 * np.cos(incline_angle)

    return ob

def apply_inclination_visual(frame, incline_angle):
    """
    Apply visual indication of inclination to the frame.
    This function adds a rotated rectangle or other visual markers to indicate the incline.
    """
    import cv2
    h, w, _ = frame.shape
    overlay = frame.copy()
    angle_degrees = np.degrees(incline_angle)

    # Draw a tilted line or rectangle to indicate the incline
    rect_center = (w // 2, h // 2)
    rect_size = (w, h // 10)
    rect = ((rect_center), rect_size, angle_degrees)

    box = cv2.boxPoints(rect)
    box = np.intp(box)  # Use np.intp instead of np.int0
    cv2.drawContours(overlay, [box], 0, (0, 255, 0), -1)

    # Blend the overlay with the original frame
    alpha = 0.4  # Transparency factor
    frame = cv2.addWeighted(overlay, alpha, frame, 1 - alpha, 0)

    return frame

In [ ]:
test_env = wrap_env(eval_env)
test_env.seed(0)

obs_nlags = 0
obs_concat_axis = -1
obs_expand_axis = None
obs_padding = 'first'

if hasattr(actor, 'obs_nlags'):
    obs_nlags = actor.obs_nlags
if hasattr(actor, 'obs_concat_axis'):
    obs_concat_axis = actor.obs_concat_axis
if hasattr(actor, 'obs_expand_axis'):
    obs_expand_axis = actor.obs_expand_axis
if hasattr(actor, 'obs_padding'):
    obs_padding = actor.obs_padding

smp = Sampler(env=test_env,
              obs_nlags=obs_nlags,
              obs_concat_axis=obs_concat_axis,
              obs_expand_axis=obs_expand_axis,
              obs_padding=obs_padding)


In [ ]:
def visualize_actor(actor, env, nsteps=200, size=(1400, 900), seed=0, train_sampling=False, data=None, incline_angle=0, case_number = 1, show_video=False, generate=True):
    # display = Display(visible=0, size=size)
    # display.start()

    # test_env = wrap_env(env)
    test_env.seed(seed)

    # obs_nlags = getattr(actor, 'obs_nlags', 0)
    # obs_concat_axis = getattr(actor, 'obs_concat_axis', -1)
    # obs_expand_axis = getattr(actor, 'obs_expand_axis', None)
    # obs_padding = getattr(actor, 'obs_padding', 'first')

    # smp = Sampler(env=test_env, obs_nlags=obs_nlags, obs_concat_axis=obs_concat_axis, obs_expand_axis=obs_expand_axis, obs_padding=obs_padding)

    # previous_mode = actor.train_sampling
    # actor.train_sampling = train_sampling

    ob = set_random_initial_state(smp, incline_angle=incline_angle,case_number=case_number)
    if show_video:
      frames = []

    for i in range(nsteps):
        if show_video:
          frame = test_env.render(mode='rgb_array')
          frames.append(frame)
        smp.add_obs(ob)
        _ob = smp.get_obs()
        _ob = np.expand_dims(_ob, axis=0) # put _ob into batchmode for consistency
        ac = actor.get_action(_ob)
        ac = ac.squeeze(axis=0) # squeeze ac back for storage
        if generate:
            qpos = env.env.sim.data.qpos
            qvel = env.env.sim.data.qvel
            data[seed, i, :] = np.concatenate([qpos, qvel]) # 18 enteries

        ob, rew, done, info = smp.step_env(ac)
        # tot_rew += rew
        # print(i)
        if done:
            break

    test_env.close()
    # actor.train_sampling = previous_mode
    if show_video:
        print('Loading video...')
        for frame in frames:
            plt.imshow(frame)
            plt.axis('off')
            plt.show()
            ipythondisplay.clear_output(wait=True)
        # show_video_fn()

In [ ]:
import matplotlib.pyplot as plt
import IPython.display as ipythondisplay
from moviepy.editor import ImageSequenceClip

def visualize_data(actor, env, nsteps=200, size=(1400, 900), seed=0, train_sampling=False, data=None, incline_angle=0, show_video=False, generate=False):
    display = Display(visible=0, size=size)
    display.start()

    test_env = wrap_env(env)
    test_env.seed(seed)

    obs_nlags = getattr(actor, 'obs_nlags', 0)
    obs_concat_axis = getattr(actor, 'obs_concat_axis', -1)
    obs_expand_axis = getattr(actor, 'obs_expand_axis', None)
    obs_padding = getattr(actor, 'obs_padding', 'first')

    smp = Sampler(env=test_env, obs_nlags=obs_nlags, obs_concat_axis=obs_concat_axis, obs_expand_axis=obs_expand_axis, obs_padding=obs_padding)

    previous_mode = actor.train_sampling
    actor.train_sampling = train_sampling

    ob = test_env.reset()

    tot_rew = 0
    frames = []

    for i in range(nsteps):
      ob = data[i, :]
      qpos = env.env.sim.data.qpos.copy()
      qvel = env.env.sim.data.qvel.copy()

      qpos[:] = data[i, :9]
      qvel[:] = data[i, 9:]
      env.env.set_state(qpos, qvel)

      frame = test_env.render(mode='rgb_array')
      frames.append(frame)

    test_env.close()
    actor.train_sampling = previous_mode
    if show_video:
        print('Loading video...')
        for frame in frames:
            plt.imshow(frame)
            plt.axis('off')
            plt.show()
            ipythondisplay.clear_output(wait=True)
        clip = ImageSequenceClip(frames, fps=30)
        clip.write_videofile(f"/content/walker_trajectory{seed}.mp4")
        print("Video saved successfully.")

In [ ]:
# To visualize the generated data:
loaded_data = torch.load('/content/Latent_ODE_ECE_228/data/HopperPhysics/training.pt')
visualize_data(actor=actor, env=eval_env, nsteps=100, train_sampling=False, data=loaded_data[100], seed=0, incline_angle=0, show_video=True, generate=False)


  self.pid = _posixsubprocess.fork_exec(



ValueError: could not broadcast input array from shape (5,) into shape (9,)

In [ ]:
!pwd

/content/relax_dyna_q_example


In [ ]:
%cd ..


/content


In [ ]:
%cd Latent_ODE_ECE_228/

/content/Latent_ODE_ECE_228


In [ ]:
! pip install torchdiffeq

In [ ]:
! pwd

/content/Latent_ODE_ECE_228


In [ ]:
import torch.optim as optim
from torch.distributions.normal import Normal
import numpy as np
import time
from random import SystemRandom

import lib.utils as utils
from lib.create_latent_ode_model import create_LatentODE_model
from lib.utils import compute_loss_all_batches, get_next_batch, makedirs, get_logger

from lib.rnn_baselines import *
from lib.ode_rnn import *
from lib.create_latent_ode_model import create_LatentODE_model
from lib.parse_datasets import parse_datasets
from lib.ode_func import ODEFunc, ODEFunc_w_Poisson
from lib.diffeq_solver import DiffeqSolver
from mujoco_physics import HopperPhysics
from lib.latent_ode import LatentODE



In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

In [ ]:
class Args:
    def __init__(self, **kwargs):
        self.__dict__.update(kwargs)

In [ ]:
args = Args(
    n=10000,  # Size of the dataset
    niters=100,
    lr=1e-2,  # Starting learning rate
    batch_size=50,
    viz=False,  # Show plots while training
    save='experiments/',  # Path to save checkpoints
    load=None,  # ID of the experiment to load for evaluation. If None, run a new experiment
    random_seed=1991,  # Random seed
    dataset='hopper',  # Dataset to load
    sample_tp=0.5,  # Number of time points to sub-sample
    cut_tp=None,  # Cut out the section of the timeline
    quantization=0.1,  # Quantization on the physionet dataset
    latent_ode=False,  # Run Latent ODE seq2seq model
    z0_encoder='odernn',  # Type of encoder for Latent ODE model
    classic_rnn=False,  # Run RNN baseline
    rnn_cell="expdecay",  # RNN Cell type #gru, expdecay- gru-d
    input_decay=True,  # For RNN: use the input that is the weighted average of empirical mean and previous value
    ode_rnn=True,  # Run ODE-RNN baseline
    rnn_vae=False,  # Run RNN baseline: seq2seq model with sampling of the h0 and ELBO loss
    latents=24,  # Size of the latent state
    rec_dims=30,  # Dimensionality of the recognition model
    rec_layers=3,  # Number of layers in ODE func in recognition ODE
    gen_layers=3,  # Number of layers in ODE func in generative ODE
    units=300,  # Number of units per layer in ODE func
    gru_units=100,  # Number of units per layer in each of GRU update networks
    poisson=False,  # Model poisson-process likelihood for the density of events in addition to reconstruction
    classif=False,  # Include binary classification loss
    linear_classif=False,  # Use a linear classifier instead of 1-layer NN
    extrap=False,  # Set extrapolation mode
    timepoints=100,  # Total number of time-points
    max_t=5.0,  # Subsample points in the interval [0, args.max_t]
    noise_weight=0.01  # Noise amplitude for generated trajectories
)


In [ ]:
torch.manual_seed(args.random_seed)
np.random.seed(args.random_seed)

experimentID = args.load
if experimentID is None:
	# Make a new experiment ID
	experimentID = int(SystemRandom().random()*100000)
ckpt_path = os.path.join(args.save, "experiment_" + str(experimentID) + '.ckpt')

start = time.time()
print("Sampling dataset of {} training examples".format(args.n))

input_command = f"run_models.py --n {args.n} --niters {args.niters} --lr {args.lr} --batch_size {args.batch_size} " \
                f"--viz {args.viz} --save {args.save} --random_seed {args.random_seed} --dataset {args.dataset} " \
                f"--latent_ode {args.latent_ode} --classic_rnn {args.classic_rnn} --ode_rnn {args.ode_rnn}--z0_encoder {args.z0_encoder} --latents {args.latents} " \
                f"--rec_dims {args.rec_dims} --rec_layers {args.rec_layers} --gen_layers {args.gen_layers} " \
                f"--units {args.units} --gru_units {args.gru_units} --timepoints {args.timepoints} --max_t {args.max_t} " \
                f"--noise_weight {args.noise_weight} --extrap {args.extrap} "

if args.load:
	input_command += f" --load {args.load}"

makedirs("results/")

print(f"Checkpoint path: {ckpt_path}")
print(f"Time taken for setup: {time.time() - start} seconds")
print(f"Input command: {input_command}")

Sampling dataset of 10000 training examples
Checkpoint path: experiments/experiment_4669.ckpt
Time taken for setup: 0.004057884216308594 seconds
Input command: run_models.py --n 10000 --niters 100 --lr 0.01 --batch_size 50 --viz False --save experiments/ --random_seed 1991 --dataset hopper --latent_ode True --classic_rnn False --ode_rnn False--z0_encoder odernn --latents 24 --rec_dims 30 --rec_layers 3 --gen_layers 3 --units 300 --gru_units 100 --timepoints 100 --max_t 5.0 --noise_weight 0.01 --extrap False 


In [ ]:
data_obj = parse_datasets(args, device)
input_dim = data_obj["input_dim"]

print(f"Input dimension: {input_dim}")

classif_per_tp = False
if ("classif_per_tp" in data_obj):
		# do classification per time point rather than on a time series as a whole
		classif_per_tp = data_obj["classif_per_tp"]

if args.classif and (args.dataset == "hopper" or args.dataset == "periodic"):
		raise Exception("Classification task is not available for MuJoCo and 1d datasets")

n_labels = 1
if args.classif:
	if ("n_labels" in data_obj):
		n_labels = data_obj["n_labels"]
	else:
		raise Exception("Please provide number of labels for classification task")

Input dimension: 18


In [ ]:
obsrv_std = 1e-3
obsrv_std = torch.Tensor([obsrv_std]).to(device)
z0_prior = Normal(torch.Tensor([0.0]).to(device), torch.Tensor([1.]).to(device))

In [ ]:
if args.rnn_vae:
		if args.poisson:
			print("Poisson process likelihood not implemented for RNN-VAE: ignoring --poisson")

		# Create RNN-VAE model
		model = RNN_VAE(input_dim, args.latents,
			device = device,
			rec_dims = args.rec_dims,
			concat_mask = True,
			obsrv_std = obsrv_std,
			z0_prior = z0_prior,
			use_binary_classif = args.classif,
			classif_per_tp = classif_per_tp,
			linear_classifier = args.linear_classif,
			n_units = args.units,
			input_space_decay = args.input_decay,
			cell = args.rnn_cell,
			n_labels = n_labels,
			train_classif_w_reconstr = (args.dataset == "physionet")
			).to(device)
elif args.classic_rnn:
		if args.poisson:
			print("Poisson process likelihood not implemented for RNN: ignoring --poisson")

		if args.extrap:
			raise Exception("Extrapolation for standard RNN not implemented")
		# Create RNN model
		model = Classic_RNN(input_dim, args.latents, device,
			concat_mask = True, obsrv_std = obsrv_std,
			n_units = args.units,
			use_binary_classif = args.classif,
			classif_per_tp = classif_per_tp,
			linear_classifier = args.linear_classif,
			input_space_decay = args.input_decay,
			cell = args.rnn_cell,
			n_labels = n_labels,
			train_classif_w_reconstr = (args.dataset == "physionet")
			).to(device)
elif args.ode_rnn:
		# Create ODE-GRU model
		n_ode_gru_dims = args.latents

		if args.poisson:
			print("Poisson process likelihood not implemented for ODE-RNN: ignoring --poisson")

		if args.extrap:
			raise Exception("Extrapolation for ODE-RNN not implemented")

		ode_func_net = utils.create_net(n_ode_gru_dims, n_ode_gru_dims,
			n_layers = args.rec_layers, n_units = args.units, nonlinear = nn.Tanh)

		rec_ode_func = ODEFunc(
			input_dim = input_dim,
			latent_dim = n_ode_gru_dims,
			ode_func_net = ode_func_net,
			device = device).to(device)

		z0_diffeq_solver = DiffeqSolver(input_dim, rec_ode_func, "euler", args.latents,
			odeint_rtol = 1e-3, odeint_atol = 1e-4, device = device)

		model = ODE_RNN(input_dim, n_ode_gru_dims, device = device,
			z0_diffeq_solver = z0_diffeq_solver, n_gru_units = args.gru_units,
			concat_mask = True, obsrv_std = obsrv_std,
			use_binary_classif = args.classif,
			classif_per_tp = classif_per_tp,
			n_labels = n_labels,
			train_classif_w_reconstr = (args.dataset == "physionet")
			).to(device)
elif args.latent_ode:
		model = create_LatentODE_model(args, input_dim, z0_prior, obsrv_std, device,
			classif_per_tp = classif_per_tp,
			n_labels = n_labels)
else:
	raise Exception("Model not specified")

In [ ]:
ckpt_path=r"/content/experiment_latents_24_ODE_RNN_0.5.ckpt"

In [ ]:
utils.get_ckpt_model(ckpt_path, model, device)

In [ ]:
test_dict = utils.get_next_batch(data_obj["test_dataloader"])

In [ ]:
data =  test_dict["data_to_predict"]
time_steps = test_dict["tp_to_predict"]
mask = test_dict["mask_predicted_data"]

observed_data =  test_dict["observed_data"]
observed_time_steps = test_dict["observed_tp"]
observed_mask = test_dict["observed_mask"]

device = utils.get_device(time_steps)

time_steps_to_predict = time_steps


if isinstance(model, LatentODE):
	# sample at the original time points
	time_steps_to_predict = utils.linspace_vector(time_steps[0], time_steps[-1], 100).to(device)

reconstructions, info = model.get_reconstruction(time_steps_to_predict,
	observed_data, observed_time_steps, mask = observed_mask, n_traj_samples = 10)

  warnings.warn("t is not on the same device as y0. Coercing to y0.device.")



In [ ]:
loaded_data[100]

array([[ 4.72591248e-03,  1.25508480e+00, -1.24619654e-03, ...,
         1.56703646e-03,  1.48397823e-04, -6.08339647e-03],
       [ 1.84401745e-03,  1.25302162e+00, -4.00697002e-02, ...,
        -1.16198611e+01,  4.80388625e-01,  1.65958729e+01],
       [-3.94614163e-03,  1.24632826e+00, -1.16791526e-01, ...,
        -1.13233929e+01,  4.29998500e-01,  3.15696615e+01],
       ...,
       [ 4.36743418e+00,  1.12175382e+00,  7.47254487e-01, ...,
        -4.05885032e-01,  1.04069116e-01, -2.28577027e+01],
       [ 4.39975970e+00,  1.13012849e+00,  7.45734361e-01, ...,
        -3.63658583e-01, -1.42319016e-01, -9.58894256e+00],
       [ 4.43165782e+00,  1.13643772e+00,  7.46991339e-01, ...,
        -2.47505941e-01, -1.14007117e-01, -5.64384175e+00]])

In [ ]:
observed_data.shape

torch.Size([2000, 100, 18])

In [ ]:
reconstructions.shape


torch.Size([1, 2000, 100, 18])

In [ ]:
reconstructions.mean(dim=0).detach()[91]

tensor([[0.2411, 0.6016, 1.7056,  ..., 0.6817, 0.5410, 1.2936],
        [0.2296, 0.6228, 1.6483,  ..., 0.6949, 0.5696, 1.2935],
        [0.2741, 0.6425, 1.6935,  ..., 0.6722, 0.5308, 1.2834],
        ...,
        [0.4834, 0.6834, 1.7255,  ..., 0.6344, 0.5093, 1.3527],
        [0.4905, 0.6856, 1.6988,  ..., 0.6496, 0.4664, 1.3366],
        [0.5001, 0.6904, 1.6767,  ..., 0.6664, 0.4184, 1.3239]],
       device='cuda:0')

In [ ]:
reconstructions.mean(dim=0).detach()[81]

tensor([[0.1239, 0.6942, 0.0754,  ..., 0.6740, 0.5338, 1.2816],
        [0.1252, 0.6950, 0.0911,  ..., 0.6514, 0.5447, 1.3209],
        [0.1281, 0.7367, 0.0340,  ..., 0.6611, 0.5209, 1.2865],
        ...,
        [0.3489, 0.6498, 1.6012,  ..., 0.6795, 0.5859, 1.1961],
        [0.3513, 0.6443, 1.6183,  ..., 0.6773, 0.5789, 1.1816],
        [0.3518, 0.6409, 1.6378,  ..., 0.6757, 0.5722, 1.1863]],
       device='cuda:0')

In [ ]:
visualize_data(actor=actor, env=eval_env, nsteps=100, train_sampling=False, data=observed_data[91].cpu(), seed=0, incline_angle=0, show_video=True, generate=False)


Moviepy - Building video /content/walker_trajectory0.mp4.
Moviepy - Writing video /content/walker_trajectory0.mp4



Moviepy - Done !
Moviepy - video ready /content/walker_trajectory0.mp4
Video saved successfully.


In [ ]:
visualize_data(actor=actor, env=eval_env, nsteps=100, train_sampling=False, data=reconstructions.mean(dim=0)[91].cpu().detach(), seed=0, incline_angle=0, show_video=True, generate=False)


Moviepy - Building video /content/walker_trajectory0.mp4.
Moviepy - Writing video /content/walker_trajectory0.mp4



Moviepy - Done !
Moviepy - video ready /content/walker_trajectory0.mp4
Video saved successfully.


# GT Render


In [ ]:
args = Args(
    n=10000,  # Size of the dataset
    niters=100,
    lr=1e-2,  # Starting learning rate
    batch_size=50,
    viz=False,  # Show plots while training
    save='experiments/',  # Path to save checkpoints
    load=None,  # ID of the experiment to load for evaluation. If None, run a new experiment
    random_seed=1991,  # Random seed
    dataset='hopper',  # Dataset to load
    sample_tp=None,  # Number of time points to sub-sample
    cut_tp=None,  # Cut out the section of the timeline
    quantization=0.1,  # Quantization on the physionet dataset
    latent_ode=True,  # Run Latent ODE seq2seq model
    z0_encoder='odernn',  # Type of encoder for Latent ODE model
    classic_rnn=False,  # Run RNN baseline
    rnn_cell="expdecay",  # RNN Cell type #gru, expdecay- gru-d
    input_decay=True,  # For RNN: use the input that is the weighted average of empirical mean and previous value
    ode_rnn=False,  # Run ODE-RNN baseline
    rnn_vae=False,  # Run RNN baseline: seq2seq model with sampling of the h0 and ELBO loss
    latents=15,  # Size of the latent state
    rec_dims=30,  # Dimensionality of the recognition model
    rec_layers=3,  # Number of layers in ODE func in recognition ODE
    gen_layers=3,  # Number of layers in ODE func in generative ODE
    units=300,  # Number of units per layer in ODE func
    gru_units=100,  # Number of units per layer in each of GRU update networks
    poisson=False,  # Model poisson-process likelihood for the density of events in addition to reconstruction
    classif=False,  # Include binary classification loss
    linear_classif=False,  # Use a linear classifier instead of 1-layer NN
    extrap=False,  # Set extrapolation mode
    timepoints=100,  # Total number of time-points
    max_t=5.0,  # Subsample points in the interval [0, args.max_t]
    noise_weight=0.01  # Noise amplitude for generated trajectories
)


In [ ]:
if args.rnn_vae:
		if args.poisson:
			print("Poisson process likelihood not implemented for RNN-VAE: ignoring --poisson")

		# Create RNN-VAE model
		model = RNN_VAE(input_dim, args.latents,
			device = device,
			rec_dims = args.rec_dims,
			concat_mask = True,
			obsrv_std = obsrv_std,
			z0_prior = z0_prior,
			use_binary_classif = args.classif,
			classif_per_tp = classif_per_tp,
			linear_classifier = args.linear_classif,
			n_units = args.units,
			input_space_decay = args.input_decay,
			cell = args.rnn_cell,
			n_labels = n_labels,
			train_classif_w_reconstr = (args.dataset == "physionet")
			).to(device)
elif args.classic_rnn:
		if args.poisson:
			print("Poisson process likelihood not implemented for RNN: ignoring --poisson")

		if args.extrap:
			raise Exception("Extrapolation for standard RNN not implemented")
		# Create RNN model
		model = Classic_RNN(input_dim, args.latents, device,
			concat_mask = True, obsrv_std = obsrv_std,
			n_units = args.units,
			use_binary_classif = args.classif,
			classif_per_tp = classif_per_tp,
			linear_classifier = args.linear_classif,
			input_space_decay = args.input_decay,
			cell = args.rnn_cell,
			n_labels = n_labels,
			train_classif_w_reconstr = (args.dataset == "physionet")
			).to(device)
elif args.ode_rnn:
		# Create ODE-GRU model
		n_ode_gru_dims = args.latents

		if args.poisson:
			print("Poisson process likelihood not implemented for ODE-RNN: ignoring --poisson")

		if args.extrap:
			raise Exception("Extrapolation for ODE-RNN not implemented")

		ode_func_net = utils.create_net(n_ode_gru_dims, n_ode_gru_dims,
			n_layers = args.rec_layers, n_units = args.units, nonlinear = nn.Tanh)

		rec_ode_func = ODEFunc(
			input_dim = input_dim,
			latent_dim = n_ode_gru_dims,
			ode_func_net = ode_func_net,
			device = device).to(device)

		z0_diffeq_solver = DiffeqSolver(input_dim, rec_ode_func, "euler", args.latents,
			odeint_rtol = 1e-3, odeint_atol = 1e-4, device = device)

		model = ODE_RNN(input_dim, n_ode_gru_dims, device = device,
			z0_diffeq_solver = z0_diffeq_solver, n_gru_units = args.gru_units,
			concat_mask = True, obsrv_std = obsrv_std,
			use_binary_classif = args.classif,
			classif_per_tp = classif_per_tp,
			n_labels = n_labels,
			train_classif_w_reconstr = (args.dataset == "physionet")
			).to(device)
elif args.latent_ode:
		model = create_LatentODE_model(args, input_dim, z0_prior, obsrv_std, device,
			classif_per_tp = classif_per_tp,
			n_labels = n_labels)
else:
	raise Exception("Model not specified")

In [ ]:
data_obj = parse_datasets(args, device)


In [ ]:
ckpt_path=  r"/content/Latent_ODE_ECE_228/experiments/experiment_10006.ckpt"

In [ ]:
utils.get_ckpt_model(ckpt_path, model, device)

In [ ]:
test_dict = utils.get_next_batch(data_obj["test_dataloader"])

In [ ]:
data =  test_dict["data_to_predict"]
time_steps = test_dict["tp_to_predict"]
mask = test_dict["mask_predicted_data"]

observed_data =  test_dict["observed_data"]
observed_time_steps = test_dict["observed_tp"]
observed_mask = test_dict["observed_mask"]

device = utils.get_device(time_steps)

time_steps_to_predict = time_steps


if isinstance(model, LatentODE):
	# sample at the original time points
	time_steps_to_predict = utils.linspace_vector(time_steps[0], time_steps[-1], 100).to(device)

reconstructions, info = model.get_reconstruction(time_steps_to_predict,
	observed_data, observed_time_steps, mask = observed_mask, n_traj_samples = 10)

In [ ]:
traj_index=350
percentage=str(args.sample_tp)
model=model.__class__.__name__

In [ ]:
output_dir= f"gt_render_{traj_index}_{percentage}_{model}"
hopper = HopperPhysics(root='data', download=False, generate=False)
hopper.visualize(observed_data[traj_index], plot_name=f'traj_{traj_index}', dirname=output_dir)
output_vid_gt=f"{output_dir}/ground_truth.mp4"
frames_to_video(output_dir,output_vid_gt)